In [ ]:
elements = driver.find_elements(By.CLASS_NAME,"body")
anchor = ''

for element in elements: 

    anchor = element.get_attribute('id')

    # Let's fetch all the title from the page 
    try:
        title = driver.find_element(By.CLASS_NAME,"text_header")
        Title = title.text
    except:
        Title = "NA"

    # Let's Fetch the user'name 
    try:
        user_name = driver.find_element(By.XPATH, f'//*[@id="{anchor}"]/h3/span/span')
        Reviewer_Name = user_name.text
    
    except:
        Reviewer_Name= "NA"

    
    # Location 

    Location = driver.find_element(By.XPATH, f'//*[@id="{anchor}"]/h3')
    loca= Location.text
    
    try:
        start_index = loca.find('(') + 1
        end_index = loca.find(')')
        country = loca[start_index:end_index]
        Country = country 
    
    except:
        Country = "NA"

    
    # Date of Posting 

    # Use regular expression to find the date pattern
    try:
        date_pattern = r'\d{1,2}(?:st|nd|rd|th) [A-Za-z]+ \d{4}'
        match = re.search(date_pattern, loca)

        if match:
            date = match.group()
            Date_of_posting = date
    
    except:
        Date_of_posting = "NA"


    # trip verified or not 
    try:
        trip = driver.find_element(By.CLASS_NAME,'text_content').text.split("|")
        trip_verified = trip[0]
        Verified = trip_verified

    except:
        Verified = "Not Found"
    

    # main review
    try: 
        main_review= trip[1]
        Review = main_review
    
    except: 
        Review = "NA"


    # Checking for the aircraft 
    try: 
        Aircraft = driver.find_element(By.XPATH, '//*[@id="{anchor}"]/div/div[2]/table/tbody/tr[1]/td[1]')
        aircraft_type = Aircraft.text

        if aircraft_type == "Aircraft":
            Aircraft =  Aircraft.text
    
    except: 
        Aircraft = "Not found "

    # Checking for the traveller 
    try: 
        type_of_traveller = driver.find_element(By.XPATH, '//*[@id="{anchor}"]/div/div[2]/table/tbody/tr[1]/td[1]')
        traveller_type = type_of_traveller.text

        if traveller_type == "Type Of Traveller	":
            traveller_type = type_of_traveller.text
    
    except: 
        traveller_type = "Not found "



In [7]:
import csv
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Function to extract star rating number from a row
def extract_star_rating(row):
    try:
        # Get the class attribute value and check if "star" and "fill" are present
        class_attribute = row.get('class')
        if class_attribute and "star" in class_attribute and "fill" in class_attribute:
            return row.text.strip()
        else:
            return 'Not Found'
    except:
        return 'Not Found'

# Function to extract value from the table row
def get_value(div_element, row_label):
    try:
        row = div_element.find('td', text=row_label)
        if row:
            value = row.find_next('td').text.strip()
            return value
        else:
            return 'Not Found'
    except:
        return 'Not Found'

# Path to the downloaded web driver executable
driver_path = '/path/to/chromedriver'  # Replace this with the actual path to the downloaded driver

# Initialize the Chrome web driver
driver = webdriver.Chrome(executable_path=driver_path)

# Load the web page containing the HTML elements
base_url = "https://www.airlinequality.com/airline-reviews/british-airways/page/1/"
driver.get(base_url)

# Get the HTML content of the page
html_content = driver.page_source

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# List to store the extracted dictionaries
review_data = []

# Find all div elements with class="body"
div_elements = soup.find_all('div', class_='body')

# Extract values from each div element and store them in the list of dictionaries
for div_element in div_elements:
    data_dict = {
        'Text Header': div_element.find('h2', class_='text_header').text.strip(),
        'User Name': div_element.find('span', itemprop='name').text.strip(),
        'Country': div_element.find('h3', class_='text_sub_header userStatusWrapper').text.split('(')[-1].replace(')', '').strip(),
        'Date Published': div_element.find('time', itemprop='datePublished').text.strip(),
        'Review Body': div_element.find('div', itemprop='reviewBody').text.strip(),
    }
    # Get star ratings for each category from 1 to 10
    for i in range(1, 11):
        try:
            label_element = div_element.select_one(f'table tbody tr:nth-of-type({i}) td:nth-of-type(1)')
            value_element = div_element.select_one(f'table tbody tr:nth-of-type({i}) td:nth-of-type(2) span[class*="star fill"]')
            if label_element and value_element:
                label = label_element.text.strip()
                value = extract_star_rating(value_element)
                data_dict[label] = value
        except:
            pass

    # Get values for Aircraft, Type Of Traveller, Seat Type, Route, Date Flown
    try:
        data_dict['Aircraft'] = get_value(div_element, 'Aircraft')
        data_dict['Type Of Traveller'] = get_value(div_element, 'Type Of Traveller')
        data_dict['Seat Type'] = get_value(div_element, 'Seat Type')
        data_dict['Route'] = get_value(div_element, 'Route')
        data_dict['Date Flown'] = get_value(div_element, 'Date Flown')
    except:
        pass

    # Get Recommended value
    try:
        recommended_value = div_element.find('div', class_='yesValue')
        if recommended_value:
            data_dict['Recommended'] = 'Yes'
        else:
            data_dict['Recommended'] = 'No'
    except:
        data_dict['Recommended'] = 'Not Found'

    review_data.append(data_dict)

# Close the web driver
driver.quit()

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(review_data)

# Save the DataFrame to a CSV file
df.to_csv('aa.csv', index=False)

# Print the DataFrame (optional)
print(df)


C:\Users\singh\AppData\Local\Temp\ipykernel_17164\575048341.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)
C:\Users\singh\AppData\Local\Temp\ipykernel_17164\575048341.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  row = div_element.find('td', text=row_label)


                                         Text Header          User Name  \
0                  "Customer Service does not exist"          N Hancock   
1                         "another great experience"     Steven Hodgson   
2                                  "Not recommended"      John Grainger   
3                  "5 hours and 30 mins but no meal"           K Robson   
4                  "Things have really deteriorated"   Pradeep Madhavan   
5              "I will never fly this airline again"       Jeffrey Rice   
6  "asked for an explanation but have received none"      Bridget Fagan   
7                        "short-changing passengers"      Bervin Hedman   
8                      "Economy is absolutely awful"  Alastair Cockburn   
9                        "Shocking customer service"          S Carlsen   

                         Country  Date Published  \
0  United Kingdom 18th July 2023  18th July 2023   
1  United Kingdom 17th July 2023  17th July 2023   
2  United Kingdom 